<a href="https://colab.research.google.com/github/navneet-jha/AIL200/blob/main/%5BSession-1%5D%20Assignment/Assignment-BERT-T5-Session_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Assignment: Fine-tuning DistilBERT for Sentiment Analysis

### 🔍 What is DistilBERT?  
DistilBERT is a **lightweight, faster, and smaller** version of BERT, designed to maintain **97% of BERT’s performance** while being **60% faster** and using **40% fewer parameters**. It achieves this through a process called **knowledge distillation**, where it learns from a larger BERT model while retaining essential knowledge.  

🚀 **Why use DistilBERT?**  
- **Faster Inference** – Great for real-time applications!  
- **Smaller Model Size** – Ideal for deployment on edge devices.  
- **Competitive Accuracy** – Performs nearly as well as BERT on many NLP tasks.  

---

__Objective__:
Fine-tune the DistilBERT model on the IMDb movie reviews dataset to classify reviews as positive or negative.

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pandas as pd
import torch

In [4]:
# Load IMDb dataset from Hugging Face
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Load the pretrained DistilBERT and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Tokenize the dataset

# Fill code here
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

0.025010755222666936


In [15]:
## Split dataset into train and test

# Fill code here
from torch.utils.data import DataLoader
train_dataset = DataLoader(tokenized_dataset["train"].select(range(1000)), shuffle=True, batch_size=32)
test_dataset = DataLoader(tokenized_dataset["test"], batch_size=32)
#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

0.27502931836911926


In [27]:
# Evaluate pre-trained DistilBERT model on IMDb test-set
# Note: Since classifier head is randomly initialized, performance is expected to be closed to random

# Fill code here
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.eval()

all_labels = []
all_predictions = []

# Set the number of batches to evaluate
num_batches_to_evaluate = 1
batch_counter = 0

# Evaluate on a limited number of batches
for batch in tqdm(test_dataset):
    if batch_counter >= num_batches_to_evaluate:
        break
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["label"]
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=1)
    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(predictions.cpu().numpy())
    batch_counter += 1

accuracy = accuracy_score(all_labels, all_predictions)
print(f"\nAccuracy: {accuracy}")

#print(classification_report(all_labels, all_predictions))
#print(confusion_matrix(all_labels, all_predictions))
#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

  0%|          | 1/782 [00:00<06:50,  1.90it/s]


Accuracy: 0.3125
0.21863797480360336


In [31]:
# Define training arguments

# Fill code here
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

0.1988376506866485


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
# Fine-tune DistilBERT model

# Fill code here
for epoch in range(3):
    for batch in tqdm(train_dataset):
        optimizer.zero_grad()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

100%|██████████| 32/32 [00:45<00:00,  1.42s/it]

0.6498844377795232


In [34]:
# Re-evaluate the fine-tuned model

# Fill code here
model.eval()

all_labels = []
all_predictions = []

# Set the number of batches to evaluate
num_batches_to_evaluate = 1
batch_counter = 0

# Evaluate on a limited number of batches
for batch in tqdm(test_dataset):
    if batch_counter >= num_batches_to_evaluate:
        break
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["label"]
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=1)
    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(predictions.cpu().numpy())
    batch_counter += 1

accuracy = accuracy_score(all_labels, all_predictions)
print(f"\nAccuracy: {accuracy}")
#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

  0%|          | 1/782 [00:00<07:08,  1.82it/s]


Accuracy: 1.0
0.2204406220406967


In [35]:
# Report the results with pre-trained model and finetuned model

# Fill code here
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Accuracy: {accuracy}")

print(classification_report(all_labels, all_predictions))
print(confusion_matrix(all_labels, all_predictions))
#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        32

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

[[32]]
0.5892656838759087


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
